<a href="https://colab.research.google.com/github/Jamoxidase/MachineLearning/blob/main/drugPointCloud01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am working on drug-protein binding inferance, specifically a large-scale dataset containing drug molocule strutures and experimental binding states for each molocule with respect to 3 proteins.

An interesting input representation of the molocule is a 3D point cloud that represents both the spacial structure of a molocule and local point charges through regional scaling.



In [4]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00


In [5]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn
import torch_geometric.data as pyg_data
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [41]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

def compute_point_cloud(smiles, num_points_per_atom_base=100 ,radius_scaling_factor=2.0):
    try:
        mol = Chem.MolFromSmiles(smiles.replace('Dy', 'H')) #<-- place holder, H is generally neutral and happy with one bond
        mol = Chem.AddHs(mol)    # Get 3D coordinates of atoms
        AllChem.EmbedMolecule(mol)
        AllChem.UFFOptimizeMolecule(mol)
        mol.GetConformer()
        AllChem.ComputeGasteigerCharges(mol) # Get point charges

        coords_list = []
        charge_list = []
        for i, atom in enumerate(mol.GetAtoms()):
            coords = mol.GetConformer().GetAtomPosition(i)
            coords_list.append(coords)

            charge = atom.GetProp('_GasteigerCharge')
            charge_list.append(charge)

        coords_list = np.array(coords_list)
        charge_list = np.array(charge_list, dtype =float)
        charge_list = charge_list.reshape(-1, 1) # 2D array for MinMaxScaler
        scaler = MinMaxScaler()
        normalized_charges = scaler.fit_transform(charge_list)
        normalized_charges = normalized_charges.flatten()
        normalized_charges = 0.3 + (1 - normalized_charges) * 0.7 #<-- skewe

        point_cloud = []
        for i in range(len(coords_list)):
            center = coords_list[i]
            radius = normalized_charges[i]

            # Dynamic point sampling for uniform density
            num_points = int(num_points_per_atom_base * (radius ** radius_scaling_factor))
            for _ in range(num_points):
                theta = 2 * np.pi * np.random.rand()  #azimuthal angle
                phi = np.arccos(2 * np.random.rand() - 1)  #polar angle

                #spherical -> cartesian coordinates
                x = center[0] + radius * np.sin(phi) * np.cos(theta)
                y = center[1] + radius * np.sin(phi) * np.sin(theta)
                z = center[2] + radius * np.cos(phi)
                point_cloud.append([x, y, z])
        point_cloud = Normalize()(np.array(point_cloud))
        return coords_list, point_cloud
    except Exception as e:
        print(f"Error with molecule {smiles}: {e}")
        return None

In [42]:
coords_list, point_cloud = compute_point_cloud("C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCC2CCC(=C)CC2)nc(Nc2ccc3c(c2)COC3=O)n1")

In [43]:
import plotly.graph_objects as go

x_coords = point_cloud[:, 0]
y_coords = point_cloud[:, 1]
z_coords = point_cloud[:, 2]

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=6,
        color=z_coords,  # set color to z values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8
    )
)])
fig.update_layout(title='3D Scatter Plot of point cloud')
# Show the plot
fig.show()

The following cell displays the atomic points in molocule. Bonds are not shown.

In [ ]:
import plotly.graph_objects as go

x_coords = coords_list[:, 0]
y_coords = coords_list[:, 1]
z_coords = coords_list[:, 2]

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=6,
        color=z_coords,  # set color to z values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8
    )
)])
fig.update_layout(title='3D Scatter Plot of coords_list')
# Show the plot
fig.show()